<a href="https://colab.research.google.com/github/trchudley/GEOG2462/blob/main/Short_Scripts/Week_2_Calculate_Median_NDIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Calculate a median-composite normalised difference index in Google Earth Engine

This script calcualtes the _median_ pixel value of all images in the imageCollection, rather than just taking the first image. This could be useful if you are examining changes over larger areas and/or longer timescales.

## Log in to Google Earth Engine

In [1]:
import ee
import geemap
import geemap.chart as chart
import time

ee.Authenticate()  # Trigger the authentication flow.
ee.Initialize(project='ee-trchudley')    # Change to your own default project name.

## Set editable parameters

Most editable parameters are in this cell. Thresholding parameters must also be manually selected after the histogram is generated.

Note that, in this example, we select between the beginning of June and end August this is to align with a climatic season (DJF, MAM, JJA, SON).

In [2]:

# Location - editable
latitude = -13.922           # Degrees of latitude
longitude = -70.821          # Degrees of longitude
size = 30000                 # Size of AOI, in metres
location_name = 'quelcayya'  # recognisable name, to create a useful file name

# Dates - editable
date_start = '2023-06-01'
date_end = '2023-08-31'

# Set NDI name and r_high and r_low bands
type_of_ndi = 'NDSI'  # type of NDI calculated, for filename purposes
r_high = 'B3'  # relevant band name for r_high
r_low = 'B6'   # relevant band name for r_low

# Google Drive export folder
folder = 'scires_project_2A'

## Search for image and calculate a median composite from all images.

In [3]:
# Set up location geometry
point = ee.Geometry.Point(longitude, latitude)  # Create a point
region = point.buffer(size/2).bounds()  # Buffer the point to a 2D shape

# Get Landsat 8 image collection
landsat8_collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA")

# Filter to desired region and date bounds
landsat8_collection = landsat8_collection.filterBounds(region)
landsat8_collection = landsat8_collection.filterDate(date_start, date_end)

# Get the least cloudy image in the collection, and clip it to our search region
image = landsat8_collection.median()
image = image.clip(region)

# # Print out date, for reference
list_length = len(landsat8_collection.toList(landsat8_collection.size()).getInfo())
print(f'Produced composite from {list_length} images')

# Calculate NDI
ndi = image.normalizedDifference([r_high, r_low]).rename(type_of_ndi)


Produced composite from 10 images


Visualise NDI for quality assessment.

In [4]:

Map = geemap.Map() # Create empty map

# Display colour image
max_reflectance = 0.80
visParams = {'bands': ['B4', 'B3', 'B2'], 'max': max_reflectance}
Map.addLayer(image, visParams, 'True Colour')

# Display NDI
visParams = {'bands': [type_of_ndi], 'min': -1, 'max': 1, 'palette': ['red', 'white', 'blue']}
Map.addLayer(ndi, visParams, type_of_ndi)

Map.centerObject(region, zoom=11)
Map


Map(center=[-13.921960750262597, -70.82079023084508], controls=(WidgetControl(options=['position', 'transparen…

# Thresholding

Use the below histogram to assess viable threshold.

In [ ]:
# Sample 10,000 pixels within the NDI image
sample_pixels = ndi.sample(region, numPixels=10000, dropNulls=True)

# Set labels for the graph
labels = {
    "title": 'Distribution of NDI values within image',
    "xlabel": f'{type_of_ndi} values',
    "ylabel": 'Pixel count',
}

print(sample_pixels.getInfo())
# # Construct the histogram
# chart.feature_histogram(sample_pixels, type_of_ndi, **labels)

{'type': 'FeatureCollection', 'columns': {'NDSI': 'Float<-1.0, 1.0>'}, 'properties': {'band_order': ['NDSI']}, 'features': []}


Manually set threshold filters below. Note the `direction` variable, which can be set to `'greater_than'` or `'less_than`' depending on whether you want to classify surfaces > or < the NDI threshold.

The `filter_threshold_km2` sets a minimum size of classified area to include. It can be set to zero if you don't want to filter.

In [ ]:
# manually set an NDI threshold with which to classify
threshold = 0.8

# Greater than or less then
direction = 'greater_than'
# direction = 'less_than'

# Set the minimum size of thresholded areas to be included, in km2. Set to zero if you don't want to filter.
filter_threshold_km2 = 0.5

The rest is automatic:

In [ ]:
# threshold the image to where greater than or less than threshold.
if direction == 'greater_than':
    ndi_threshold = ndi.gte(threshold)
elif direction == 'less_than':
    ndi_threshold = ndi.lte(threshold)
else:
    raise ValueError("`direction` variable must be set to 'greater_than' or 'less_than'")

# 'Mask' the data, showing only regions beyond the threshold.
ndi_threshold = ndi_threshold.updateMask(ndi_threshold.neq(0))

# Calculate the area of each pixel in km2, and add it as a band to the NDI image
ndi_and_area = ndi_threshold.addBands(ndi_threshold.multiply(30*30).divide(1e6))

# Use the `reduceToVectors()` function to produce vectors, also calculating
# the total area using the `sum()` function to sum the pixel areas.
vectors = ndi_and_area.reduceToVectors(
  scale=30,
  geometryType = 'polygon',
  eightConnected = False,
  reducer = ee.Reducer.sum(),
  maxPixels=1e9,
)

# Extra line to rename the area column to `area_km2`.
vectors = vectors.map(lambda feature: feature.set('area_km2', feature.get('sum')).set('sum', None).set('label', None))

# Filter polygons smaller than this chosen threshold
vectors_filtered = vectors.filter(ee.Filter.gte('area_km2', filter_threshold_km2))


Visualise threshold for quality assessment. The unfiltered thresholded region is included as a cyan raster; the area-filtered region is included as a blue vector.

In [ ]:

Map = geemap.Map() # Create empty map

# Display colour image
max_reflectance = 0.80
visParams = {'bands': ['B4', 'B3', 'B2'], 'max': max_reflectance}
Map.addLayer(image, visParams, 'True Colour')

# Display thresholded NDI as raster
visParams = {'bands': [type_of_ndi], 'palette': ['white', 'cyan'], 'opacity': 0.3}
Map.addLayer(ndi_threshold, visParams, type_of_ndi)

# Display vectors filtered
Map.addLayer(vectors_filtered, {'color': 'blue'}, "Identified Region")  # Add our AOI

Map.centerObject(region, zoom=12)
Map


Map(center=[-13.921836156025476, -70.82048134334799], controls=(WidgetControl(options=['position', 'transparen…

Print quantiative statistics:

In [ ]:

# Get total area of initial search region
aoi_area_km2 = region.area(maxError=1).getInfo() / 1e6

# Get total area of all vectors
vector_area_km2 = vectors.aggregate_sum('area_km2').getInfo()

# Get total area the filtered vectors
vector_flt_area_km2 = vectors_filtered.aggregate_sum('area_km2').getInfo()

# Print the results
print(f'Total scene area: {aoi_area_km2:.2f} km2')
print(f'Total classified area: {vector_area_km2:.2f} km2')
print(f'Total classified area (filtered): {vector_flt_area_km2:.2f} km2')


## Download data

Download initial scene:

In [ ]:
# Construct the filename automatically
date_string = image.get('DATE_ACQUIRED').getInfo()
filename = location_name + '_' + date_start + '_' date_start + '_median_image'

# Print out filename for reference
print("The image will be saved to your Google Drive at:\n" + folder + '/' + filename + '.tif\n')

# Export the image, specifying scale and region.
task = ee.batch.Export.image.toDrive(**{
    'image': image.select(['B4', 'B3', 'B2', 'B5', 'B6']),
    'description': filename,
    'folder': folder,
    'scale': 30,
    'region': region.getInfo()['coordinates']
})
task.start()

while task.active():
  print('Task processing ongoing... (id: {}).'.format(task.id))
  time.sleep(5)

print('Finished processing. Image is exported to your Drive.')


Download NDI:

In [ ]:
# Construct the filename automatically
filename = location_name + '_' + date_start + '_' date_start + '_median_' + type_of_ndi

# Print out filename for reference
print("The image will be saved to your Google Drive at:\n" + folder + '/' + filename + '.tif\n')

# Export the image, specifying scale and region.
task = ee.batch.Export.image.toDrive(**{
    'image': ndi,
    'description': filename,
    'folder': folder,
    'scale': 30,
    'region': region.getInfo()['coordinates']
})
task.start()

while task.active():
  print('Task processing ongoing... (id: {}).'.format(task.id))
  time.sleep(5)

print('Finished processing. Image is exported to your Drive.')


Download vector data:

In [ ]:
# Construct the filename automatically
date_string = image.get('DATE_ACQUIRED').getInfo()
filename = location_name + '_' + date_string + '_' + type_of_ndi + '_theshold_' + str(max_reflectance)

# Print out filename for reference
print("The image will be saved to your Google Drive at:\n" + folder + '/' + filename + '.kml\n')

# Export the featureCollection, specifying scale and region.
task = ee.batch.Export.table.toDrive(**{
  'collection': vectors_filtered,
  'description': filename,
  'folder': folder,
  # 'fileFormat': 'KML',
  'fileFormat': 'SHP',
})
task.start()

while task.active():
  print('Task processing ongoing... (id: {}).'.format(task.id))
  time.sleep(5)

print('Finished processing. Vector file is exported to your Drive.')
